In [53]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import numbers
import decimal
import sys
import os
import datetime

In [54]:
# download data

os.system("wget -N https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

os.system("wget -N https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

os.system("wget -N https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")


0

In [55]:
confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('time_series_covid19_recovered_global.csv')

print(confirmed.head())

colnames = confirmed.columns.tolist()
print(colnames)

  Province/State Country/Region      Lat     Long  1/22/20  1/23/20  1/24/20  \
0            NaN    Afghanistan  33.0000  65.0000        0        0        0   
1            NaN        Albania  41.1533  20.1683        0        0        0   
2            NaN        Algeria  28.0339   1.6596        0        0        0   
3            NaN        Andorra  42.5063   1.5218        0        0        0   
4            NaN         Angola -11.2027  17.8739        0        0        0   

   1/25/20  1/26/20  1/27/20  ...  3/21/20  3/22/20  3/23/20  3/24/20  \
0        0        0        0  ...       24       40       40       74   
1        0        0        0  ...       76       89      104      123   
2        0        0        0  ...      139      201      230      264   
3        0        0        0  ...       88      113      133      164   
4        0        0        0  ...        2        2        3        3   

   3/25/20  3/26/20  3/27/20  3/28/20  3/29/20  3/30/20  
0       84       94   

In [57]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days+1)]

print(confirmed.shape)
print(date_generated)

(254, 73)
[datetime.datetime(2020, 1, 22, 0, 0), datetime.datetime(2020, 1, 23, 0, 0), datetime.datetime(2020, 1, 24, 0, 0), datetime.datetime(2020, 1, 25, 0, 0), datetime.datetime(2020, 1, 26, 0, 0), datetime.datetime(2020, 1, 27, 0, 0), datetime.datetime(2020, 1, 28, 0, 0), datetime.datetime(2020, 1, 29, 0, 0), datetime.datetime(2020, 1, 30, 0, 0), datetime.datetime(2020, 1, 31, 0, 0), datetime.datetime(2020, 2, 1, 0, 0), datetime.datetime(2020, 2, 2, 0, 0), datetime.datetime(2020, 2, 3, 0, 0), datetime.datetime(2020, 2, 4, 0, 0), datetime.datetime(2020, 2, 5, 0, 0), datetime.datetime(2020, 2, 6, 0, 0), datetime.datetime(2020, 2, 7, 0, 0), datetime.datetime(2020, 2, 8, 0, 0), datetime.datetime(2020, 2, 9, 0, 0), datetime.datetime(2020, 2, 10, 0, 0), datetime.datetime(2020, 2, 11, 0, 0), datetime.datetime(2020, 2, 12, 0, 0), datetime.datetime(2020, 2, 13, 0, 0), datetime.datetime(2020, 2, 14, 0, 0), datetime.datetime(2020, 2, 15, 0, 0), datetime.datetime(2020, 2, 16, 0, 0), datetime.d

In [58]:
def pred_function(series, k=1):
    # Newton-Leibniz baseline -- local prediction by derivative
    # In 1665, following an outbreak of the bubonic plague in England, 
    # Cambridge University closed its doors, forcing Newton to return home and 
    # develop Differential calculus.
    # Leibniz's most prominent accomplishment was conceiving the ideas of differential and integral calculus, 
    # independently of Isaac Newton's contemporaneous developments.
    # Local prediction by derivative
    # Discrete backward derivative delta(x) = f(x) - f(x-1)
    # f(x+1)= delta(x)+f(x)
    
    delta = series[-1] - series[-2]
    pred = k*delta + series[-1]
    return pred

In [59]:

num_day_pred = 2
last_date = date_generated[-1]

last_date_str = last_date.strftime("%Y-%m-%d")
next_pred_date = last_date+datetime.timedelta(days=num_day_pred)
file_str = str(num_day_pred)+"day_prediction_" + last_date_str + ".csv"
print(file_str)

2day_prediction_2020-03-30.csv


137

In [68]:

print(confirmed.shape[0])

f = open(file_str,"w+")

f.write("Province/State,Country,Target/Date,N,low95N,high95N,R,low95R,high95R,D,low95D,high95D,T,low95T,high95T,M,low95M,high95M,C,low95C,high95C\n")

province_null_idx = confirmed['Province/State'].isnull()


for idx in range(0,confirmed.shape[0]):

    try:
        if (province_null_idx[idx] == True):
            loc1_str = ","
        else:
            loc1 = confirmed['Province/State'].iloc[idx]
            loc1_str = str(loc1).replace(',', ' ') + ","


        loc2 = confirmed['Country/Region'].iloc[idx]
        loc2_str = str(loc2).replace(',', ' ') + ","
        print(loc2_str)

        try:
            next_confirmed = pred_function(confirmed.iloc[idx][colnames[4:]].to_numpy(),2)
            n_str = str(next_confirmed)+",,,"
        except:
            next_confirmed = float("NaN")
            n_str = ",,,"

        try:
            next_deaths = pred_function(deaths.iloc[idx][colnames[4:]].to_numpy(),2)
            d_str = str(next_deaths) + ",,,"
        except:
            next_deaths = float("NaN")
            d_str = ",,,"

        try:
            next_recovered = pred_function(recovered.iloc[idx][colnames[4:]].to_numpy(),2)
            r_str = str(next_recovered)+",,,"
        except:
            next_recovered = float("NaN")
            r_str = ",,,"

        if (next_confirmed > 0):
            if (next_deaths>0):
                next_mortality = next_deaths / next_confirmed 
                m_str = str(next_mortality)+",,,"
            else:
                m_str = "0,,,"
        else:
            next_mortality = float("NaN")
            m_str = ",,,"

        next_pred_date_str = next_pred_date.strftime("%Y-%m-%d")+","
        t_str = ",,,"
        c_str = ",,\n"

        try:
            f.write(loc1_str+loc2_str+next_pred_date_str+n_str+r_str+d_str+t_str+m_str+c_str)
        except:
            print("CAN NOT WRITE to file\n")
    except:
        print("Unexpected error:", sys.exc_info()[0])

f.close()
print("END")

254
Afghanistan,
Albania,
Algeria,
Andorra,
Angola,
Antigua and Barbuda,
Argentina,
Armenia,
Australia,
Australia,
Australia,
Australia,
Australia,
Australia,
Australia,
Australia,
Austria,
Azerbaijan,
Bahamas,
Bahrain,
Bangladesh,
Barbados,
Belarus,
Belgium,
Benin,
Bhutan,
Bolivia,
Bosnia and Herzegovina,
Brazil,
Brunei,
Bulgaria,
Burkina Faso,
Cabo Verde,
Cambodia,
Cameroon,
Canada,
Canada,
Canada,
Canada,
Canada,
Canada,
Canada,
Canada,
Canada,
Canada,
Canada,
Central African Republic,
Chad,
Chile,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
China,
Colombia,
Congo (Brazzaville),
Congo (Kinshasa),
Costa Rica,
Cote d'Ivoire,
Croatia,
Diamond Princess,
Cuba,
Cyprus,
Czechia,
Denmark,
Denmark,
Denmark,
Djibouti,
Dominican Republic,
Ecuador,
Egypt,
El Salvador,
Equatorial Guinea,
Eritrea,
Estonia,
Eswatini,
Eth